In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2.extras as extras
import pycountry

In [10]:
# Stáhnutí příslušných souborů CSV
tables = pd.read_html("https://www.kurzy.cz/svet/", na_values=["-", "–", "—"])
# Získání tabulky
df1 = tables[0]
# Získání názvů sloupců
print(df1)
# Přejmenování sloupců
df1 = df1.rename(columns={
    "Stát": "country",
    "Rok": "year",
    "CZK": "GDP"
})
# Výběr sloupců, které jsou relevantní
df1 = df1[["country", "year", "GDP"]]
# kontrola dat
print(df1)

                       Stát   Rok         USD          CZK  Zpráv
0    Spojené státy americké  2023  27 720 709  615 676 947  7 372
1                      Čína  2023  17 794 783  395 222 131  1 187
2                   Německo  2023   4 525 704  100 515 884  1 627
3                  Japonsko  2023   4 204 495   93 381 830    293
4                     Indie  2023   3 567 552   79 235 323    181
..                      ...   ...         ...          ...    ...
203                   Palau  2023         282        6 260      0
204                Kiribati  2023         279        6 201      0
205     Marshallovy ostrovy  2023         259        5 759      1
206                   Nauru  2023         154        3 424      0
207                  Tuvalu  2023          62        1 383      0

[208 rows x 5 columns]
                    country  year          GDP
0    Spojené státy americké  2023  615 676 947
1                      Čína  2023  395 222 131
2                   Německo  2023  100 515 

In [11]:
# Vytvoření slovníku pro mapování českých názvů zemí na ISO kódy
# Získání unikátních jmen zemí z datasetu
country_names = df1['country'].unique()
print(f"Počet unikátních zemí v datasetu: {len(country_names)}")

# Vytvoření slovníku pro mapování českých názvů zemí na ISO kódy
country_mapping = {
    'Spojené státy americké': 'USA',
    'Čína': 'CHN',
    'Německo': 'DEU',
    'Japonsko': 'JPN',
    'Indie': 'IND',
    'Velká Británie': 'GBR',
    'Francie': 'FRA',
    'Rusko': 'RUS',
    'Brazílie': 'BRA',
    'Itálie': 'ITA',
    'Kanada': 'CAN',
    'Jižní Korea': 'KOR',
    'Austrálie': 'AUS',
    'Španělsko': 'ESP',
    'Mexiko': 'MEX',
    'Indonésie': 'IDN',
    'Nizozemsko': 'NLD',
    'Nizozemí': 'NLD',
    'Saúdská Arábie': 'SAU',
    'Švýcarsko': 'CHE',
    'Polsko': 'POL',
    'Turecko': 'TUR',
    'Švédsko': 'SWE',
    'Belgie': 'BEL',
    'Thajsko': 'THA',
    'Irsko': 'IRL',
    'Rakousko': 'AUT',
    'Norsko': 'NOR',
    'Nigérie': 'NGA',
    'Izrael': 'ISR',
    'Argentina': 'ARG',
    'Jihoafrická republika': 'ZAF',
    'Dánsko': 'DNK',
    'Singapur': 'SGP',
    'Česko': 'CZE',
    'Česká republika': 'CZE',
    'Spojené arabské emiráty': 'ARE',
    'Portugalsko': 'PRT',
    'Rumunsko': 'ROU',
    'Katar': 'QAT',
    'Peru': 'PER',
    'Řecko': 'GRC',
    'Kazachstán': 'KAZ',
    'Alžírsko': 'DZA',
    'Maďarsko': 'HUN',
    'Kuvajt': 'KWT',
    'Ukrajina': 'UKR',
    'Malajsie': 'MYS',
    'Vietnam': 'VNM',
    'Finsko': 'FIN',
    'Bangladéš': 'BGD',
    'Pákistán': 'PAK',
    'Egypt': 'EGY',
    'Chile': 'CHL',
    'Kolumbie': 'COL',
    'Filipíny': 'PHL',
    'Nový Zéland': 'NZL',
    'Slovensko': 'SVK',
    'Maroko': 'MAR',
    'Ekvádor': 'ECU',
    'Omán': 'OMN',
    'Bělorusko': 'BLR',
    'Ázerbájdžán': 'AZE',
    'Bulharsko': 'BGR',
    'Srbsko': 'SRB',
    'Guatemala': 'GTM',
    'Chorvatsko': 'HRV',
    'Lucembursko': 'LUX',
    'Dominikánská republika': 'DOM',
    'Panama': 'PAN',
    'Uruguay': 'URY',
    'Keňa': 'KEN',
    'Kostarika': 'CRI',
    'Jordánsko': 'JOR',
    'Slovinsko': 'SVN',
    'Angola': 'AGO',
    'Libanon': 'LBN',
    'Turkmenistán': 'TKM',
    'Litva': 'LTU',
    'Tádžikistán': 'TJK',
    'Tunisko': 'TUN',
    'Bolívie': 'BOL',
    'Ghana': 'GHA',
    'Bahrajn': 'BHR',
    'Uzbekistán': 'UZB',
    'Lotyšsko': 'LVA',
    'Etiopie': 'ETH',
    'Paraguay': 'PRY',
    'Trinidad a Tobago': 'TTO',
    'Honduras': 'HND',
    'Kypr': 'CYP',
    'Estonsko': 'EST',
    'Papua-Nová Guinea': 'PNG',
    'Salvador': 'SLV',
    'Venezuela': 'VEN',
    'Zambie': 'ZMB',
    'Myanmar': 'MMR',
    'Afghánistán': 'AFG',
    'Island': 'ISL',
    'Bosna a Hercegovina': 'BIH',
    'Severní Makedonie': 'MKD',
    'Malta': 'MLT',
    'Laos': 'LAO',
    'Mongolsko': 'MNG',
    'Senegal': 'SEN',
    'Gruzie': 'GEO',
    'Nikaragua': 'NIC',
    'Mauritius': 'MUS',
    'Mosambik': 'MOZ',
    'Uganda': 'UGA',
    'Kyrgyzstán': 'KGZ',
    'Botswana': 'BWA',
    'Jamajka': 'JAM',
    'Namibie': 'NAM',
    'Kambodža': 'KHM',
    'Brunej': 'BRN',
    'Kongo': 'COG',
    'Rwanda': 'RWA',
    'Haiti': 'HTI',
    'Nepál': 'NPL',
    'Čad': 'TCD',
    'Moldavsko': 'MDA',
    'Albánie': 'ALB',
    'Togo': 'TGO',
    'Arménie': 'ARM',
    'Madagaskar': 'MDG',
    'Benin': 'BEN',
    'Guyana': 'GUY',
    'Bahamy': 'BHS',
    'Černá Hora': 'MNE',
    'Maledivy': 'MDV',
    'Fidži': 'FJI',
    'Svazijsko': 'SWZ',
    'Eswatini': 'SWZ',
    'Surinam': 'SUR',
    'Bhútán': 'BTN',
    'Barbados': 'BRB',
    'Jižní Súdán': 'SSD',
    'Sierra Leone': 'SLE',
    'Džibutsko': 'DJI',
    'Rovníková Guinea': 'GNQ',
    'Maldivská republika': 'MDV',
    'Východní Timor': 'TLS',
    'Antigua a Barbuda': 'ATG',
    'Burundi': 'BDI',
    'Seychely': 'SYC',
    'Belize': 'BLZ',
    'Svatá Lucie': 'LCA',
    'Kapverdy': 'CPV',
    'Středoafrická republika': 'CAF',
    'Eritrea': 'ERI',
    'Libérie': 'LBR',
    'Somálsko': 'SOM',
    'Guinea-Bissau': 'GNB',
    'Šalamounovy ostrovy': 'SLB',
    'Guinea': 'GIN',
    'Grenada': 'GRD',
    'Lesotho': 'LSO',
    'Vanuatu': 'VUT',
    'Gambie': 'GMB',
    'Komory': 'COM',
    'Svatý Vincenc a Grenadiny': 'VCT',
    'Saint Kitts a Nevis': 'KNA',
    'Samoa': 'WSM',
    'Svatý Tomáš a Princův ostrov': 'STP',
    'Dominika': 'DMA',
    'Mikronésie': 'FSM',
    'Tonga': 'TON',
    'Palau': 'PLW',
    'Kiribati': 'KIR',
    'Marshallovy ostrovy': 'MHL',
    'Nauru': 'NRU',
    'Tuvalu': 'TUV',
    'Írán': 'IRN',
    'Hongkong': 'HKG',
    'ČR': 'CZE',
    'Irák': 'IRQ',
    'Portoriko': 'PRI',
    'Súdán': 'SDN',
    'Srí Lanka': 'LKA',
    'Kuba': 'CUB',
    'Tanzanie': 'TZA',
    'Pobřeží slonoviny': 'CIV',
    'Barma': 'MMR',
    'Kongo (Kinshasa)': 'COD',
    'Kamerun': 'CMR',
    'Bahrain': 'BHR',
    'Macao': 'MAC',
    'Libye': 'LBY',
    'Zimbabwe': 'ZWE',
    'Sýrie': 'SYR',
    'Jemen': 'YEM',
    'Mali': 'MLI',
    'Burkina Faso': 'BFA',
    'Gabon': 'GAB',
    'Stát Palestina': 'PSE',
    'Niger': 'NER',
    'Moldávie': 'MDA',
    'Kongo (Brazzaville)': 'COG',
    'Brunei': 'BRN',
    'Malawi': 'MWI',
    'Mauritánie': 'MRT',
    'Kosovo': 'XKX',
    'Monako': 'MCO',
    'Nová Kaledonie': 'NCL',
    'Bermudy': 'BMU',
    'Man (ostrov)': 'IMN',
    'Lichtejnštejnsko': 'LIE',
    'Kajmanské ostrovy': 'CYM',
    'Guam': 'GUM',
    'Francouzská Polynésie': 'PYF',
    'Americké Panenské ostrovy': 'VIR',
    'Faerské ostrovy': 'FRO',
    'Andorra': 'AND',
    'Aruba': 'ABW',
    'Grónsko': 'GRL',
    'Kapverské ostrovy': 'CPV',
    'San Marino': 'SMR',
    'Svatý Vincent': 'VCT',
    'Svatý Kryštof a Nevis': 'KNA',
    'Americká Samoa': 'ASM',
    'Sv. Tomáš a Princův ostrov': 'STP',
    'Dominica': 'DMA',
    'Federativní státy Mikronésie': 'FSM'
}

# Aplikace mapování na sloupec country
df1['country_iso'] = df1['country'].map(country_mapping)

# Kontrola, zda některé země nebyly namapovány
missing_mapping = df1[df1['country_iso'].isna()]
if len(missing_mapping) > 0:
    print(f"Počet zemí bez mapování: {len(missing_mapping)}")
    print("Seznam zemí bez mapování:")
    print(missing_mapping["country"].unique())
else:
    print("Všechny země byly úspěšně namapovány na ISO kódy!")

# Nahrazení původního sloupce country ISO kódy
df1['original_country'] = df1['country']  # Uložení původních názvů pro pozdější referenci
df1['country'] = df1['country_iso']
df1.drop('country_iso', axis=1, inplace=True)

# Zobrazení výsledku
print(df1.head(10))

Počet unikátních zemí v datasetu: 208
Všechny země byly úspěšně namapovány na ISO kódy!
  country  year          GDP        original_country
0     USA  2023  615 676 947  Spojené státy americké
1     CHN  2023  395 222 131                    Čína
2     DEU  2023  100 515 884                 Německo
3     JPN  2023   93 381 830                Japonsko
4     IND  2023   79 235 323                   Indie
5     GBR  2023   75 088 779          Velká Británie
6     FRA  2023   67 781 180                 Francie
7     ITA  2023   51 103 903                  Itálie
8     BRA  2023   48 277 114                Brazílie
9     CAN  2023   47 584 279                  Kanada


In [17]:
# Vytvořit z df soubor csv
df1 = df1[["country", "year", "GDP"]]
# Uložení do CSV
df1.to_csv("gdp_data.csv", index=False)